In [2]:
'''Trains a LSTM on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function
import numpy as np
import scipy.io
import h5py

np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers.core import Flatten
from keras.layers import LSTM, SimpleRNN, GRU,Convolution1D,MaxPooling1D
from keras.datasets import imdb
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD,RMSprop



Using Theano backend.


In [3]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')

validmat = scipy.io.loadmat('/Users/jx/Documents/CS273B/validlstm.mat')
testmat = scipy.io.loadmat('/Users/jx/Documents/CS273B/testlstm.mat')

trainmat = scipy.io.loadmat('/Users/jx/Documents/CS273B/trainlstm.mat')

X_train=trainmat['tr'][0][0][0]
y_train=trainmat['tr'][0][0][1]

X_valid=validmat['v'][0][0][0]
y_valid=validmat['v'][0][0][1]

X_test=testmat['tt'][0][0][0]
y_test=testmat['tt'][0][0][1]

print X_train.shape()
print y_train.shape()

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

SyntaxError: invalid syntax (<ipython-input-3-1a05b5c1efc4>, line 20)

In [14]:

batch_size = 128


print('Build model...')
model = Sequential()
# model.add(Embedding(max_features, 128, dropout=0.2))

model.add(LSTM(640, dropout_W=0.2, dropout_U=0.2, input_dim=4))  # try using a GRU instead, for fun
# model.add(Convolution1D(64, 4, border_mode = 'same',input_shape=(4,111)))
# # model.add(Dense(1))
# # model.add(Activation('tanh'))
# model.add(Activation('relu'))

# model.add(MaxPooling1D(pool_length=(4)))

# model.add(Flatten)
model.add(Dense(input_dim=111, output_dim=1))
model.add(Activation('softmax'))
# model.add(forward_lstm)
# model.add(brnn)
rmsprop=RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])




Build model...


In [15]:
print('Train...')

checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)


model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer,earlystopper])


Train...
Train on 8440 samples, validate on 1055 samples
Epoch 1/15
8440/8440 [==============================] - 153s - loss: 7.9466 - acc: 0.5015 - val_loss: 8.1601 - val_acc: 0.4882
Epoch 2/15
8440/8440 [==============================] - 156s - loss: 7.9466 - acc: 0.5015 - val_loss: 8.1601 - val_acc: 0.4882
Epoch 3/15
8440/8440 [==============================] - 156s - loss: 7.9466 - acc: 0.5015 - val_loss: 8.1601 - val_acc: 0.4882
Epoch 4/15
8440/8440 [==============================] - 160s - loss: 7.9466 - acc: 0.5015 - val_loss: 8.1601 - val_acc: 0.4882
Epoch 5/15
8440/8440 [==============================] - 153s - loss: 7.9466 - acc: 0.5015 - val_loss: 8.1601 - val_acc: 0.4882
Epoch 00004: early stopping


In [ ]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
